1. Connect to elastic search.
2. Pull data one by one
3. use openai to summarize
4. post the summary into the summary key of each index

In [28]:
from elasticsearch import Elasticsearch
import openai
import os

In [29]:
client = Elasticsearch(
        ['https://localhost:9200'],
        basic_auth=('elastic', 'elastic'),
        verify_certs = False
    )

In [30]:
response = client.search(
        index="brave_project",
        body={"query": {"match_all": {}}},
        size=1000,
        scroll="2m"  # Keep the scroll context valid for 2 minutes
    )

C:\Users\segor\AppData\Local\Temp\ipykernel_22796\1995549113.py:1: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = client.search(
C:\Users\segor\miniconda3\envs\brave\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [31]:
documents = response['hits']['hits']
# for doc in documents:
#     print(f"Document ID: {doc['_id']}")
#     print(f"Document Content: {doc['_source']}")
#     print("---------------")

In [32]:
OPENAI_API_KEY = # your api key

In [33]:
import openai
import os
from openai import OpenAI
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [78]:
jobs = response['hits']['hits']

for job in jobs:
    job_desc = job['_source']['description']
    
    # Create a chat completion
    chat_response = openai_client.chat.completions.create(
        model="gpt-4o",  # Specify the GPT-4 model
        messages=[
            {
                "role": "system",
                "content": "You are an assistant that summarizes job descriptions into concise bullet points."
            },
            {
                "role": "user",
                "content": f"Summarize the following job description:\n\n{job_desc}"
            }
        ],
        max_tokens=100  # Limit the response length
    )
    
    # Extract the summary from the assistant's response
    summary = chat_response.choices[0].message.content
    # print(summary)
    # break
    client.update(
            index="brave_project",
            id=job['_id'],
            body={"doc": {"summary": summary}}
        )
    print(job['_id'],job["_source"]["location"],job["_source"]["title"])
    



0 Toronto Business / Data Analyst
1 Toronto Data Reporting Analyst (2214)
2 Toronto Senior Campaign Data Analyst
3 Toronto Senior Data Analyst
4 Toronto BI/Data Analyst
5 Toronto Lead Data Analyst
6 Toronto DATA AND PROGRAM EVALUATION ANALYST
7 Toronto Data Analyst, Fraud Threat Management
8 Toronto Data Analytics Senior Analyst
9 Toronto Investment Data Analyst
10 Toronto Data Scientist (Global Data and Analytics)
11 Toronto Lead Data Scientist, Next Best Action
12 Toronto Catalyst Program 2025 - Data Scientist
13 Toronto Lead Data Scientist
14 Toronto Data Scientist I
15 Toronto Senior Data Scientist
16 Toronto Scientist - Data Intelligence (Remote)
17 Toronto Full Stack Data Scientist - Assistant Vice President
18 Toronto Data Scientist II
19 Toronto Data Scientist, Decisions - Driver Segments
20 Toronto Senior Product & Delivery manager
21 Toronto Product Manager, Ad Platforms
22 Toronto Senior Product Manager
23 Toronto Investment Product Manager - Bilingual (English & Spanish Req

In [62]:
print(type(chat_response.choices[0].message.content))

<class 'str'>


In [72]:
print(type(job["_source"]["description"]))

<class 'str'>
